# Flame temperature matching development
This notebook is for the development of functions required to match the adiabatic flame temperatures, $T_{ad}$, of different diluents. $T_{ad}$ matching will be used to estimate purely the thermal effect of a chemically active diluent, such as $CO_{2}$. In order to do this, the same undiluted mixture will be diluted with a relatively inert species, such as $N_{2}$, such that $T_{ad, CO_{2}} \approx T_{ad, N_{2}}$. *This will only account for the thermal effect on the detonation behavior*; sound-speed effects will be accounted for by normalizing measured wave speeds by the Chapman-Jouguet wave speed, $D_{CJ}$ for the appropriate mixture.

In [1]:
import cantera as ct
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize

In [11]:
mech = "gri30.cti"
init_temp = 300
init_press = ct.one_atm
fuel = "C3H8"
oxidizer = "O2:1 N2:3.76"
ϕ = 1
dil_active_mol_frac = 0.02
dil_active = "CO2"
dil_inert = "N2"

In [3]:
def diluted_species_dict(spec, diluent, diluent_mol_frac):
    if diluent not in spec.keys():
        spec = {k: v * (1 - diluent_mol_frac) for k, v in spec.items()}
        spec[diluent] = diluent_mol_frac
        return spec
    else:
        spec[diluent] += 1 / (1 / diluent_mol_frac - 1)
        new_total_moles = sum(spec.values())
        for s in spec.keys():
            spec[s] /= new_total_moles
        return spec

In [29]:
def get_T_ad(
    mech, 
    fuel, 
    oxidizer,
    ϕ,
    diluent, 
    diluent_mol_frac,
    init_temp, 
    init_press
):
    """
    Calculates the adiabatic flame temperature of a given mixture using
    Cantera
    
    Parameters
    ----------
    mech : str
        Mechanism to use
    fuel : str
        Fuel to use; must be in `mech`
    oxidizer : str
        Oxidizer to use; must be in `mech`
    ϕ : float
        Equivalence ratio
    diluent: str
        Species with which to dilute the mixture; must be in `mech`
    diluent_mol_frac : float
        Mole fraction of active diluent to apply to the undiluted mixture
    init_temp : float
        Mixture initial temperature
    init_press : float
        Mixture initial pressure
    
    Returns
    -------
    float
        Adiabatic flame temperature of the input mixture in Kelvin
    """
    gas = ct.Solution(mech)
    gas.set_equivalence_ratio(ϕ, fuel, oxidizer)
    gas.TPX = (
        init_temp, 
        init_press, 
        diluted_species_dict(
            gas.mole_fraction_dict(),
            diluent,
            diluent_mol_frac
        )
    )
    gas.equilibrate("HP")
    return gas.T

In [28]:
def temp_error(
    diluent_mol_frac,
    target_temp,
    mech,
    fuel,
    oxidizer,
    ϕ,
    diluent, 
    init_temp, 
    init_press
):
    """
    Compares the adiabatic flame temperature from a given combination of
    inputs to a target temperature and returns the absolute value of the
    resulting difference.
    
    Parameters
    ----------
    diluent_mol_frac : float
        Mole fraction of active diluent to apply to the undiluted mixture
    target_temp : float
        Adiabatic flame temperature to match, in Kelvin
    mech : str
        Mechanism to use
    fuel : str
        Fuel to use; must be in `mech`
    oxidizer : str
        Oxidizer to use; must be in `mech`
    ϕ : float
        Equivalence ratio
    diluent: str
        Diluent with which to evaluate the new adiabatic flame temperature;
        must be in `mech`
    init_temp : float
        Mixture initial temperature
    init_press : float
        Mixture initial pressure
        
    Returns
    -------
    float
        Absolute difference between the target temperature and the adiabatic
        flame temperature of the input mixture, in Kelvin
    """
    return abs(
        get_T_ad(
            mech=mech, 
            fuel=fuel,
            oxidizer=oxidizer,
            ϕ=ϕ, 
            diluent=diluent,
            diluent_mol_frac=diluent_mol_frac,
            init_temp=init_temp, 
            init_press=init_press
        ) - target_temp
    )

In [27]:
def match_T_ad(
    mech, 
    fuel,
    oxidizer,
    ϕ, 
    dil_active,
    dil_active_mol_frac,
    dil_inert,
    init_temp, 
    init_press,
    tol=1e-6
):
    """
    This function returns the **additional** mole fraction of a diluent gas
    required to match the adiabatic flame temperature of another diluent. If
    the diluent is *not* in the original mixture (e.g. H2/O2 diluted with N2)
    this will be the **total** mole fraction; if the diluent **is** in the
    original mixture (e.g. H2/air diluted with N2) then the **total** mole
    fraction can be seen by calling:
    
    diluted_species_dict(
        gas.mole_fraction_dict(),
        dil_inert,
        inert_mol_frac
    )
    
    The **additional** mole fraction is returned because, in this application,
    air is being added as a single component, and thus the partial pressure
    of the **additional** nitrogen is a parameter of interest.
    
    Parameters:
    -----------
    mech : str
        Mechanism to use
    fuel : str
        Fuel to use; must be in `mech`
    oxidizer : str
        Oxidizer to use; must be in `mech`
    ϕ : float
        Equivalence ratio of undiluted mixture
    dil_active : str
        Active diluent, which gives the target adiabatic flame temperature
        to be matched; must be in `mech`
    dil_active_mol_frac : float
        Mole fraction of active diluent to apply to the undiluted mixture
    dil_inert : str
        Inert diluent to match to the active diluent; must be in `mech`
    init_temp : float
        Mixture initial temperature
    init_press : float
        Mixture initial pressure
    tol : float
        Tolerance for adiabatic flame temperature matching, in Kelvin
        
    Returns
    -------
    float
        Additional mole fraction of diluent gas needed to match the adiabatic
        flame temperature to within the specified tolerance
    """
    target_temp = get_T_ad(
        mech, 
        fuel,
        oxidizer,
        ϕ, 
        dil_active,
        dil_active_mol_frac,
        init_temp, 
        init_press
    )
    best = minimize(
        temp_error, 
        [dil_active_mol_frac], 
        args=(
            target_temp,
            mech, 
            fuel,
            oxidizer,
            ϕ, 
            dil_inert,
            init_temp, 
            init_press
        ),
        method="Nelder-Mead",
        tol=tol
    )
    return best.x[0]

In [30]:
inert_mol_frac = match_T_ad(
    mech,
    fuel,
    oxidizer,
    ϕ,
    dil_active,
    dil_active_mol_frac,
    dil_inert,
    init_temp,
    init_press
)
inert_mol_frac

0.037018741607666036

In [31]:
get_T_ad(
    mech, 
    fuel, 
    oxidizer,
    ϕ,
    dil_inert, 
    inert_mol_frac,
    init_temp, 
    init_press
) - get_T_ad(
    mech, 
    fuel, 
    oxidizer,
    ϕ,
    dil_active, 
    dil_active_mol_frac,
    init_temp, 
    init_press
)

9.196974133374169e-07

In [32]:
diluted_species_dict(
    gas.mole_fraction_dict(),
    dil_inert,
    inert_mol_frac
)

{'C3H8': 0.038829889451303785,
 'N2': 0.7670206632921772,
 'O2': 0.19414944725651892}